In [337]:
import tweepy
import plotly
import datetime

import matplotlib as plt
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from plotly.subplots import make_subplots
from datetime import timedelta

In [3]:
# Keys
access_token = '1258532548134797313-w24ae6Fc7uIHO0jXk40akPLrFfUvMW'

access_token_secret = 'L8PwULD3RHMYHnxc7cJlvZSYtXTQORc0ElEFIfL1S7V21'

consumer_key = 'YuJdN38YPbdwFJpAUooTHhGbh'

consumer_key_secret = 'IOzsqk0Vly9Y90xhXqXQqQ1tsyLS11zgVTjGnpdMTkYJzKH2Dl'

# Start the API 
auth = tweepy.OAuthHandler(consumer_key,consumer_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [ ]:
tweets_list = []

for t in tweepy.Cursor(api.user_timeline, id = 'GGallegos24').items():
    try:
        # Create dictionary for specific follower and save data in there
        tweet = {}
        tweet['id'] = t.id
        tweet['text'] = t.text

        tweet['hashtags'] = t.entities['hashtags']
        tweet['user_mentions'] = t.entities['user_mentions']
        
        tweet['source'] = t.source

        tweet['created_at'] = t.created_at    

    except:
        pass
    
    tweets_list.append(tweet)


In [186]:
df = pd.DataFrame(tweets_list)
print(df.head(6))

id                                               text  \
0  1273385539094315009  RT @JuliaDeGallego1: @GGallegos24 Gracias mi v...   
1  1273378509390479361     Gracias por tanto amor https://t.co/CB45Cv2Iny   
2  1273369476512329728  Este malnacido es un mercenario https://t.co/p...   
3  1273368247870672896  Aquí está otro hijo de Guanaba https://t.co/pT...   
4  1273353797579026433  Aquí está el Tata de ARENA y FMLN. https://t.c...   
5  1273265820932718592  RT @cancilleriasv: ¡Traemos a casa a un nuevo ...   

  hashtags                                      user_mentions  \
0       []  [{'screen_name': 'JuliaDeGallego1', 'name': 'J...   
1       []                                                 []   
2       []                                                 []   
3       []                                                 []   
4       []                                                 []   
5       []  [{'screen_name': 'cancilleriasv', 'name': 'RRE...   

                source  

In [309]:
# Create Custom Columns

## RT
def rt_count(text):
    if text[0:2] == 'RT':
        return('RT')
    else:
        return('Original Tweet')
df['RT_Status'] = df['text'].apply(rt_count)

## Parse Source
def parse_source(source):
    index = (len(source)-7)
    return(source[index:])

df['source'] = df['source'].apply(parse_source)

## Parse user_mention
mentions = list(df['user_mentions'])

for i in range(len(mentions)):
    try:
        df.loc[i,'user_mention'] = mentions[i][0]['screen_name']
        df.loc[i,'user_mention_name'] = mentions[i][0]['name']
    except:
        df.loc[i,'user_mention'] = ''
        df.loc[i,'user_mention_name'] = ''

## Parse Hashtags
hashtags = list(df['hashtags'])

for i in range(len(mentions)):
    try:
        df.loc[i,'hashtag'] = hashtags[i][0]['text']
    except:
        df.loc[i,'hashtag'] = ' '

## Parse dates
def parse_day(created_at):
    return(created_at.day)
def parse_month(created_at):
    return(created_at.month)
def parse_year(created_at):
    return(created_at.year)
def parse_weekday(created_at):
    return(created_at.strftime('%A'))

## Parse hours
def parse_hour(created_at):
    return(created_at.hour)

df['day'] = df['created_at'].apply(parse_day)
df['month'] = df['created_at'].apply(parse_month)
df['year'] = df['created_at'].apply(parse_year)
df['hour'] = df['created_at'].apply(parse_hour)
df['weekday'] = df['created_at'].apply(parse_weekday)


print(df.head(5))

id                                               text  \
0  1273385539094315009  RT @JuliaDeGallego1: @GGallegos24 Gracias mi v...   
1  1273378509390479361     Gracias por tanto amor https://t.co/CB45Cv2Iny   
2  1273369476512329728  Este malnacido es un mercenario https://t.co/p...   
3  1273368247870672896  Aquí está otro hijo de Guanaba https://t.co/pT...   
4  1273353797579026433  Aquí está el Tata de ARENA y FMLN. https://t.c...   

  hashtags                                      user_mentions   source  \
0       []  [{'screen_name': 'JuliaDeGallego1', 'name': 'J...  Android   
1       []                                                 []  Android   
2       []                                                 []  Android   
3       []                                                 []  Android   
4       []                                                 []  Android   

           created_at       RT_Status     user_mention user_mention_name  \
0 2020-06-17 22:42:25              R

In [292]:
# PLOTS
## Most common hashtags
df_2 = df.groupby('hashtag').count().reset_index()
### Drop the first value because that is the value of empty string
df_2.drop(0, inplace=True)
### Order values by count
df_2 = df_2.sort_values(by = 'hashtags', ascending = False)
### Select top 10
df_2 = df_2.head(10)
fig = px.bar(
    df_2,
    y = 'hashtag',
    x = 'text',
    orientation='h',
    title = 'Hashtag: Top 10',
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()


In [282]:
## Most common mentioned users
df_2 = df.groupby('user_mention_name').count().reset_index()
### Drop the first value because that is the value of empty string
df_2.drop(0, inplace=True)
### Order values by count
df_2 = df_2.sort_values(by = 'text', ascending = False)
### Select top 10
df_2 = df_2.head(10)
fig = px.bar(
    df_2,
    y = 'user_mention_name',
    x = 'text',
    orientation='h',
    title = 'Mentioned Users: Top 10',
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()

In [272]:
## Percent Original Tweets vs Retweets
## Most common mentioned users
df_2 = df.groupby('RT_Status').count().reset_index()
### Order values by count
df_2 = df_2.sort_values(by = 'text', ascending = False)

fig = px.pie(
    df_2,
    values = 'text',
    names = 'RT_Status',
    title = 'Percent Retweets vs Original Tweets',
)
fig.show()

In [375]:
df_2 = df.groupby(['weekday','hour']).count().reset_index()

def order_weekday(weekday):
    if weekday == 'Monday':
        return(7)
    elif weekday == 'Tuesday':
        return(6)
    elif weekday == 'Wednesday':
        return(5)
    elif weekday == 'Thursday':
        return(4)
    elif weekday == 'Friday':
        return(3)
    elif weekday == 'Saturday':
        return(2)
    elif weekday == 'Sunday':
        return(1)
df_2['sort_weekday'] = df_2['weekday'].apply(order_weekday)

df_2 = df_2.sort_values(by = 'sort_weekday')
fig = px.scatter(
            df_2,
            y = 'weekday', 
            x = 'hour', 
            color = 'text', 
            size = 'text',
            marginal_x = 'violin',
            marginal_y = 'violin')
fig.show()

In [377]:
fig = go.Figure(go.Heatmap(y = df_2['weekday'],x = df_2['hour'], z = df_2['text'], colorscale = 'viridis'))

fig.show('png');